In [14]:
import pandas as pd
import numpy as np
import fiona
from shapely.geometry import shape, Polygon, MultiPolygon, Point, MultiPoint
import cPickle as pickle

In [2]:
df = pd.read_pickle('df_first757_cleaned.pkl')

In [3]:
# Read in shapefile
shapefilename = 'data/Neighborhoods'
shp = fiona.open(shapefilename+'.shp')

In [4]:
# Get neighborhood polys and their indices
polys = [shape(pol['geometry']) for pol in shp]
shp.close()


In [5]:
neighborhood_index = [i+1 for i in range(len(polys))]

In [6]:
# Associate each neighborhood polygon with its index
neighborhoods_tup = []
for hood in xrange(len(polys)):
    neighborhoods_tup.append((MultiPolygon(polys)[hood],\
    neighborhood_index[hood]))

In [7]:
# Get potholes
all_potholes = MultiPoint([Point(x, y) for x, y in zip(df['longitude'],\
    df['latitude'])])

In [8]:
# Associate each pothole with its df index
potholes_tup = []
for hole in xrange(df.shape[0]):
    potholes_tup.append((all_potholes[hole], df.index.tolist()[hole]))

In [9]:
# Keep the potholes, indices that fall within the neighborhoods
city_holes_inds = []
city_holes = []
for hole in xrange(len(potholes_tup)):
    for hood in xrange(len(neighborhoods_tup)):
        if neighborhoods_tup[hood][0].contains(potholes_tup[hole][0]):
            city_holes.append(potholes_tup[hole][0])
            city_holes_inds.append(potholes_tup[hole][1])

In [15]:
city_potholes = (city_holes, city_holes_inds)
with open('city_potholes.pkl', 'w') as f:
        pickle.dump(city_potholes, f)

In [10]:
# Extract the neighborhood index associated with each pothole
neighborhood_label = []
for hole in xrange(len(city_holes)):
    for hood in xrange(len(neighborhoods_tup)):
        if neighborhoods_tup[hood][0].contains(city_holes[hole]):
            neighborhood_label.append(neighborhoods_tup[hood][1])


In [11]:
# Add labels to dataframe
df['neighborhood_label'] = pd.Series(neighborhood_label,\
    index = city_holes_inds)

In [12]:
df.head()

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_label
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42
2,3,2649,32nd Ave NE and NE 135th ...,NE 135TH ST BETWEEN 32ND AVE NE AND LAKE CITY ...,2010-03-31 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,1 days,1,47.707446,-122.301566,"Lake City Way NE, Seattle, WA, USA","10754-10850 Lake City Way NE, Seattle, WA 9812...",104
3,4,1591,Charles st & 6 Ave S pothole next to traffic l...,S CHARLES ST BETWEEN 6TH AVE S AND MAYNARD AVE...,2010-03-15 07:00:00,2010-03-16 07:00:00,2010-03-16 07:00:00,1 days,1,47.597085,-122.325065,"Maynard Ave S, Seattle, WA, USA","608 Maynard Ave S, Seattle, WA 98104, USA",61
4,5,1680,N 80th St between Corliss Ave N and 2nd Ave NE...,N 80TH ST BETWEEN CORLISS (W LEG) AVE N AND CO...,2010-03-16 07:00:00,2010-03-17 07:00:00,2010-03-17 07:00:00,1 days,1,47.606209,-122.332071,"Seattle, WA, USA","909 5th Ave, Seattle, WA 98164, USA",62
